# **Introduction**

This notebook is for implementing the base SARSA algorithm, an on-policy temporal difference method. Similar to other implementations, this will be done using the Frozen Lake environment offered through Gymnasium. Gymnasium is an open source Python library for developing and comparing reinforcement learning algorithms, through the use of a standardized API. The `Env` class is leveraged within Gymnasium to encapsulate an environment and the Markov Decision Process (MDP) that runs behind the scenes.

# **Import Packages**

This section imports the necessary packages.

In [ ]:
# Import these packages:
import gymnasium as gym
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt